# 練習網址：http://www.chinabooktrading.com/song/
## 目標
爬取目錄，以及照階層排好。
## 步驟
1. 設定requests。
2. 設定user-agent池，以便替換。
3. 解析目錄層級。
4. 爬蟲。

In [1]:
#!/usr/bin/env python
#coding=utf-8

In [2]:
# 建立各種 HTTP 請求，從網頁伺服器上取得想要的資料
import requests

# 用來解析處理requests取得的數據; processing XML and HTML
#from lxml import html

# 一個HTML解析器，可以處理各種奇怪的HTML
from bs4 import BeautifulSoup as bs

# 睡眠用
from time import sleep

# 處理javascript用
import json

# 能處理載入、整理與視覺化等常見的資料應用套件
import pandas as pd

# 隨機睡眠用
import random

import os

#偽造User-Agent
from fake_useragent import UserAgent

import re

print('套件載入完成')

套件載入完成


### 設定requests

In [3]:
# user-agent池
UA_pool = [] #空池
ua = UserAgent() # 設定User-Agent()

for i in range(100): # 先取50個偽User-Agent就好
    UA_pool.append(ua.random) # 偽User-Agent加入UA_pool

# 隨機生成UA的函式
def UA():  
    num = random.randint(0, 100-1) 
    UA = UA_pool[num]
    headers = {'User-Agent': UA} #ua.random
    return headers

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/Users/TAI/.local/share/virtualenvs/WebCrawler-Q8hI92k6/lib/python3.7/site-packages/fake_useragent/utils.py", line 67, in get
    context=context,
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/usr/local/Cella

In [4]:
s = requests.Session()
headers = UA()

# 得cookies
# 透過開發者工具(F12)分析得知目錄的連結為'http://www.chinabooktrading.com/song/tree.jsp'
## 第一位 # #全宋詩 > 卷一至卷五八 > 全宋詩卷一 > 宋太祖 > 日詩 and 句
res = s.get('http://www.chinabooktrading.com/song/tree.jsp?e=1&book=1&column=1&poet=1', headers = headers) 

cookies = s.cookies.get_dict()
# print(s.cookies.get_dict())

In [ ]:
# 解析網頁
html = bs(res.text)
print(html)

In [ ]:
# 縮小範圍
html = html.find("div")
html

In [ ]:
## 最後一位 # #全宋詩 > 卷三七三七至卷三七八二 > 卷三七三七至卷三七八二 > 葉升 > 登凌雲山絕頂 
res2 = s.get('http://www.chinabooktrading.com/song/tree.jsp?e=1&book=72&column=3782&poet=9204', headers = UA()) 

# 解析網頁 
html2 = bs(res2.text)

# 縮小範圍
html2 = html2.find("div")
html2

透過

第一位作者之網址: 'http://www.chinabooktrading.com/song/tree.jsp?e=1&book=1&column=1&poet=1'

最後一位作者網址: 'http://www.chinabooktrading.com/song/tree.jsp?e=1&book=72&column=3782&poet=9204'

得知e , book, colum, 及 poet之變動規則

### 目錄層級順序
得卷號：href=re.compile("e={num}&".format(num = i)) 

得詩卷號：href=re.compile("book={num}&".format(num = i))

得作者：href=re.compile("column={num}&".format(num = i))

得詞名：html.find_all("a", target="main") 

===========================================================
### 變數
名稱：name, ex.全宋詩

卷號：no, ex. 卷一至卷五八

詩卷號：poemNo, ex. 全宋詩卷一

作者：author, ex. 宋太祖

詞名：poemName, ex. 日詩

In [ ]:
# URL
URL = "http://www.chinabooktrading.com/song/tree.jsp"

# 試做字典
final_dic = [] # 總字典

# 設定基本參數
id_ = 0
book_no = 0
poet_no = 0 

# 詩名
name = "全宋詩"

URL_book = URL + '?e=1' # 設定號URL
res = s.get(URL_book, headers = UA(), timeout= 5) # requests
html = bs(res.text) # 解析網頁 
book_list = html.find_all("a", href=re.compile("e=1&")) # 得全部的卷號名

for book in range(0, len(book_list)): 
    no = book_list[book].text # 得卷號名
    # print("設定卷號名: ", no)
    
    URL_column = "http://www.chinabooktrading.com/song/" + book_list[book]['href'] # 設定詩卷號URL
    #print(URL_column)
    res = s.get(URL_column, headers = UA(), timeout= 5) # requests  
    html = bs(res.text) # 解析網頁  
    
    book_no = book+1
    book_keyword = URL_column.replace(URL_book,"")+"&" # URL_column - URL_book 得關鍵字
    column_list = html.find_all("a", href=re.compile(book_keyword)) # 得全部的詩卷號 
              
    for column in range (len(column_list)): 
        poemNo = column_list[column].text # 得詩卷號
        # print("詩卷號: ", poemNo)

        URL_poet = "http://www.chinabooktrading.com/song/" + column_list[column]['href'] # 設定作者名URL  
        #print(URL_poet)
        res = s.get(URL_poet, headers = UA(), timeout= 5)
        html = bs(res.text) # 解析網頁 
        
        column_no = column+1
        column_keyword = URL_poet.replace(URL_column,"")+"&" # URL_poet - URL_column 得關鍵字
        poet_list = html.find_all("a", href=re.compile(column_keyword)) # 得全部的作者 
        
        for poet in poet_list:
            
            author = poet.text # 得作者名

            URL_poem = "http://www.chinabooktrading.com/song/" + poet['href']
            res = s.get(URL_poem, headers = UA(), timeout= 5)
            html = bs(res.text) # 解析網頁 
            poem = html.find_all("a", target="main") # 得全部的詞
            
            for poem_Num in range(0, len(poem)):
                poemName = poem[poem_Num].text # 得詞名
                # print("詞名: ", poemName)
                
                URL_poem = "http://www.chinabooktrading.com/song/" + poem[poem_Num]['href']
                #print(URL_poem)
                
                id_ += 1 # 設定id
                
                # 目錄字典
                index_dic = {
                    "id": id_,
                    "詩名": name, 
                    "卷號": no,
                    "詩卷號": poemNo,
                    "作者": author,
                    "詞名": poemName,
                    "網址": URL_poem
                     }
                print("目錄:", index_dic)
                
                # 加入總字典
                final_dic.append(index_dic)
                #print("")

print("")
print("="*80)                
print("Done!")

### 存檔

In [6]:
## 存檔
def save():
    print("**存檔中...") # 檢查哨
    dn = "ouptput" # 設立output資料夾
    file = "SongShi_index" # 宋詞檔名 # i == 資料筆數
    #error_file = "error_list" # 錯誤連結清單檔名

    try:
        if not os.path.exists(dn):   
            os.makedirs(dn)
    except:
        pass

    # 存json格式
    ## 宋詞
    f = open(dn + "/"+ file + ".json", "w", encoding="utf-8") # 打開路徑
    json.dump(final_dic, f) # 寫入json格式
    f.close() # 關閉路徑
    
    ## 錯誤清單
    """
    e = open(dn + "/"+ error_file + ".json", "w", encoding="utf-8") # 打開路徑
    json.dump(error_list, e) # 寫入json格式
    e.close() # 關閉路徑
    """
    print("json存檔完成") # 檢查哨
    
    
    # 存csv格式
    ## 宋詞
    final_dic_df = pd.DataFrame(final_dic)
    final_dic_df.to_csv(dn + "/"+ file + ".csv",encoding="utf-8", index=False)
    
    ## 錯誤清單
    """
    error_list_df = pd.DataFrame(error_list)
    error_list_df.to_csv(dn + "/"+ error_file + ".csv",encoding="utf-8", index=False)
    """
    print("csv存檔完成") # 檢查哨
    print("-" * 80)
    print("-" * 80)

    # DONE
    print("存檔完成") # 檢查哨

In [8]:
save()

**存檔中...
json存檔完成
csv存檔完成
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
存檔完成
